In [1]:
# !nvidia-smi

# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
# CUDA = format_cuda_version(CUDA_version)

In [2]:
import json
import os
# import uuid  # https://docs.python.org/3/library/uuid.html
# import structlog  # for event logging
# # from dotenv import load_dotenv # enviornment vars if we want

# from pygtail import Pygtail
# import boto3
# from minio import Minio
# from dotenv import load_dotenv
from datetime import timedelta, datetime
from time import sleep
from sys import argv
# import threading
# from smart_open import smart_open
import gensim
import gensim.corpora as corpora

# setup to import the preprocessor
import sys
# from botocore.errorfactory import ClientError
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
import torch_geometric
from functools import reduce
from tqdm import tqdm

from Preprocessing import Preprocessor
preprocessor = Preprocessor(0)
tqdm.pandas()


c:\Python310\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
c:\Python310\lib\site-packages\joblib\backports.py:7: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.version import LooseVersion


In [3]:
joined_exists = os.path.exists('khan_joined.csv')
khan = pd.DataFrame([])
if joined_exists:
    khan = pd.read_csv('khan_joined.csv')
khan.head(5)

,course,unit,lesson,video_title,about,transcript,transcript_n_entries
0,Computer programming,Intro to JS: Drawing & Animation,Intro to programming,What is Programming?,Programming is the process of creating a set o...,"['welcome', 'programming', 'learn', 'program',...",1423
1,Computer programming,Intro to JS: Drawing & Animation,Coloring,The Power of the Docs,Created by Pamela Fox.,"['voiceover', 'a few program', 'wonder', 'eart...",4018
2,Computer programming,Intro to HTML/CSS: Making webpages,Further learning,HTML validation,Learn how to validate your webpages with the W...,"['voiceover', 'khan academy', 'the oh no', 'gu...",1188
3,Computer programming,Intro to SQL: Querying and managing data,SQL basics,Welcome to SQL,SQL is useful for creating and querying relati...,"['- instructor', 'the world', 'datum', 'datum'...",1721
4,Computer programming,Intro to SQL: Querying and managing data,SQL basics,S-Q-L or SEQUEL?,How is it pronounced? Why? Let's discuss...,"['this point', 'probably', 'hear', 'pronounce'...",932


In [4]:
if not joined_exists:
    computing = pd.read_csv("Datasets\\KhanAcademy\\Computing.csv")
    computing = computing.dropna()
    computing.info()

In [5]:
if not joined_exists:
    economics = pd.read_csv("Datasets\\KhanAcademy\\Economics.csv")
    economics = economics.dropna()
    economics.info()

In [6]:
if not joined_exists:
    humanities = pd.read_csv("Datasets\\KhanAcademy\\Humanities.csv")
    humanities = humanities.dropna()
    humanities.info()

In [7]:
if not joined_exists:
    math = pd.read_csv("Datasets\\KhanAcademy\\Math.csv")
    math = math.dropna()
    math.info()

In [8]:
if not joined_exists:
    science = pd.read_csv("Datasets\\KhanAcademy\\Science.csv")
    science = science.dropna()
    science.info()

In [9]:
ted_main = pd.read_csv("Datasets\\TEDTalksDataset\\ted_main.csv")
transcripts = pd.read_csv("Datasets\\TEDTalksDataset\\transcripts.csv")
validation = ted_main.join(transcripts, lsuffix='url', rsuffix='url', sort=True)
validation = validation.dropna()
validation.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2461 entries, 0 to 2466
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   comments            2461 non-null   int64 
 1   description         2461 non-null   object
 2   duration            2461 non-null   int64 
 3   event               2461 non-null   object
 4   film_date           2461 non-null   int64 
 5   languages           2461 non-null   int64 
 6   main_speaker        2461 non-null   object
 7   name                2461 non-null   object
 8   num_speaker         2461 non-null   int64 
 9   published_date      2461 non-null   int64 
 10  ratings             2461 non-null   object
 11  related_talks       2461 non-null   object
 12  speaker_occupation  2461 non-null   object
 13  tags                2461 non-null   object
 14  title               2461 non-null   object
 15  urlurl              2461 non-null   object
 16  views               2461

In [10]:
if not joined_exists:
    computing['course'].value_counts()

In [11]:
if not joined_exists:
    economics['course'].value_counts()

In [12]:
if not joined_exists:
    humanities['course'].value_counts()

In [13]:
if not joined_exists:
    math['course'].value_counts()

In [14]:
if not joined_exists:
    science['course'].value_counts()

In [15]:
if not joined_exists:
    khan_dfs = [computing, economics, humanities, math, science]
    khan = pd.concat(khan_dfs, axis=0)
    khan.info()

In [16]:
if not joined_exists:
    # remap the courses to more broad categories: https://stackoverflow.com/a/16476974
    labels = ['physics', 'chemistry', 'biology', 'algebra', 'geometry', 'statistics', 'calculus', 'history', 'macroeconomics', 'microeconomics']
    for lbl in labels:
        for index, row in khan.iterrows():
            if lbl in row['course'].lower():
                row['course'] = lbl
    khan['course'].value_counts()

In [17]:
if not joined_exists:
    khan.tail(2)

# Dataset Identification
It is necessary to now identify components of the dataset that can be used for the graphical machine learning. This means identifying Nodes, Edges, Node Features, and Labels. It also includes optionally including edge weights and edge features. For the sake of simplicity I think that I will be forgoing the edge weights and edge features.

I am attempting to basically do topic modeling, but without the keywords and topics that would be customary of Latent Dirchlet Allocation. Instead the goal is to train a Graph ML model using the Khan academy data.

There are two basic routes I could take. I could perform node level prediction by treating each transcript individually and seeing which is the closest match during prediction. The other option would be to perform graph level prediction by storing all of the similarly labeled transcripts together and then using the shape of the graph for comparison. 

- Nodes (Items, People, Locations, Cars, ETC)
- Edges (Connections, Interactions, Similarity, ETC)
    - Levenshtein distance over titles?
    - Number of similar named entities from SpaCy NER?
- Node Features (Attributes)
- Labels (Node-Level, Edge-Level, Graph-Level, etc)
    - I am going to first try node-level prediction as it makes more sense to me. For this I am going to use the 'course' feature in the above pandas dataframe. This will be the target that I try to predict.

In [18]:
if not joined_exists:
    # It takes 21 min to run SpaCy preprocessing over each record in the data
    khan['transcript_cleaned'] = khan['transcript'].progress_apply(lambda x: preprocessor.clean(x))
    print(khan['transcript_cleaned'][0])

In [47]:
def str_to_list(s: str) -> list:
    s = s.replace('[', '')
    s = s.replace(']', '')
    s = s.replace('\'', '')
    s = s.split(', ')
    return s

In [48]:
if not joined_exists:
    # clear up the lists to be unique only.
    khan['transcript_cleaned'] = khan['transcript'].progress_apply(lambda x: list(set(str_to_list(x))))
    print(len(khan['transcript_cleaned'][0]))
    print(khan['transcript_cleaned'][0])

100%|██████████| 8261/8261 [00:00<00:00, 11492.19it/s]

86
['tell', 'my favorite robot', 'give', 'the app', 'believe', 'other programming language', 'khan academy', 'cure', 'actually', 'game', 'water', 'listen', 'look', 'possible', 'self-drive car', 'think', 'your favorite movie', 'the website', 'such cool special effect', 'course', 'patient', 'google map', 'learn', 'a drawing program', 'cool', 'wonder', 'important', 'a weird form', 'a very obedient dog', 'wikipedia', 'angry bird minecraft program', 'a great start', 'not the same basic concept', 'doctor', 'every program', 'every programmer', 'pixar', 'javascript', 'know', 'disease', 'mar', 'youtube', 'turn--', 'jump', 'process', 'really fun game', 'depend', 'create', 'easily', 'robot', 'write', 'harry potter', 'the surface program', 'welcome', 'programming', 'care', 'able', 'the computer', 'doodle', 'your every command', 'huge amount', 'pretty', 'your head', 'help', 'javascript program', 'relate', 'medical datum program', 'movie', 'a series', 'like', 'a program', 'command', 'what programmin

In [53]:
if not joined_exists:
# if not False:
    khan['transcript_n_entries'] = khan['transcript_cleaned'].progress_apply(lambda x: len(x))
    khan.head(5)

100%|██████████| 8261/8261 [00:00<00:00, 634413.82it/s]


In [20]:
# khan = khan.drop(columns=["Unnamed: 0.1", "Unnamed: 0"])

In [54]:
if not joined_exists:
# if not False:
    khan.to_csv("khan_joined.csv", index=False)

In [22]:
# https://www.kaggle.com/code/pavansanagapati/knowledge-graph-nlp-tutorial-bert-spacy-nltk/notebook

"""
    nodes closer to yourself are more important. Take the incoming embeddings and dotproduct them with your own embedding.
    Softmax across all the dot products of the nodes coming in to have it be a probability.
    
    Need to build the corpus of all input words.
    
    Use spacy to get corpus for each doc with stopwords stripped and bigrams / trigrams joined. 
    
    Could use skipgram or bag of words with word2vec to embed words.
"""

'\n    nodes closer to yourself are more important. Take the incoming embeddings and dotproduct them with your own embedding.\n    Softmax across all the dot products of the nodes coming in to have it be a probability.\n    \n    Need to build the corpus of all input words.\n    \n    Use spacy to get corpus for each doc with stopwords stripped and bigrams / trigrams joined. \n    \n    Could use skipgram or bag of words with word2vec to embed words.\n'

In [55]:
khan.head(5)

,course,unit,lesson,video_title,about,transcript,transcript_n_entries,transcript_cleaned
0,Computer programming,Intro to JS: Drawing & Animation,Intro to programming,What is Programming?,Programming is the process of creating a set o...,"['welcome', 'programming', 'learn', 'program',...",86,"[tell, my favorite robot, give, the app, belie..."
1,Computer programming,Intro to JS: Drawing & Animation,Coloring,The Power of the Docs,Created by Pamela Fox.,"['voiceover', 'a few program', 'wonder', 'eart...",205,"[line, the docs machine, example, finally, the..."
2,Computer programming,Intro to HTML/CSS: Making webpages,Further learning,HTML validation,Learn how to validate your webpages with the W...,"['voiceover', 'khan academy', 'the oh no', 'gu...",75,"[try, the w3 validation service, the html spec..."
3,Computer programming,Intro to SQL: Querying and managing data,SQL basics,Welcome to SQL,SQL is useful for creating and querying relati...,"['- instructor', 'the world', 'datum', 'datum'...",103,"[an item, example, column, a badge table, datu..."
4,Computer programming,Intro to SQL: Querying and managing data,SQL basics,S-Q-L or SEQUEL?,How is it pronounced? Why? Let's discuss...,"['this point', 'probably', 'hear', 'pronounce'...",58,"[our video, your query, figure, like, example,..."


In [25]:
# khan['transcript_cleaned'] = khan['transcript']

In [49]:
corpus = khan['transcript_cleaned'].sum(axis=0)
if isinstance(corpus, str):
    # means it performed string concatenation so it needs to be cleaned up
    print(corpus[0:100])
    corpus = str_to_list(corpus)
    print(corpus[0: 25])
    print(len(corpus))
    print(type(corpus))

In [52]:
# convert the corpus to a set since there should be no unique values
corpus = set(corpus)
print(len(corpus))
corpus = list(corpus)
print(corpus[0:25])

272850
['', 'then this molecule', 'not a procedural one', 'just the angle', 'an irishman', 'little pie piece', 'the actual remainder', '2sin', '"158 centimeter pritis height eric"', 'kind of a quick way', 'any single direction', 'the prophase', 'another notch', 'not all palestinians', 'other military structure', 'another resonance form', 'these two culture', 'a legitimate g', 'overweight', 'all us asset', '"anyas rate 1a deck"', 'a remarkably early image', 'my own body', '"benfords law vi hart"', 'a different transaction']


In [67]:
num_topics = len(khan['course'].value_counts())
print(num_topics)

26


In [62]:
%%time
# LDA Normal
temp = [d.split() for d in corpus]
print(type(temp))
words = corpora.Dictionary(temp)
words.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
corpus = [words.doc2bow(doc) for doc in temp]

<class 'list'>
CPU times: total: 3.14 s
Wall time: 3.15 s


In [68]:
%%time
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=words,
                                           num_topics=num_topics,
                                           random_state=2,
                                           update_every=1,
                                           passes=15,
                                           alpha='auto')

CPU times: total: 5min 18s
Wall time: 5min 23s


In [69]:
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, words, mds='mmds', R=10)

c:\Python310\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [70]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.510488 -0.022516       1        1  13.149147
25    -0.506490 -0.056412       2        1   8.878735
11     0.213872  0.048482       3        1   6.332498
5     -0.117465 -0.013250       4        1   4.947943
0     -0.445246 -0.237306       5        1   4.605268
20     0.103125 -0.205241       6        1   4.192167
12    -0.471725  0.122680       7        1   4.145266
4      0.318142 -0.187724       8        1   4.014497
6      0.467672 -0.200840       9        1   3.723514
23     0.478871  0.169589      10        1   3.649015
14     0.352336  0.374184      11        1   3.387925
9      0.166134 -0.458197      12        1   3.375089
15    -0.332376 -0.351752      13        1   3.288444
17    -0.212142 -0.428855      14        1   3.144113
22    -0.223875  0.239716      15        1   2.896745
18    -0.416622  0.284136      16        1   2.773263
10     0.021698  0.202749      17        1   2.754828
21    -0.060191 -0.343885      18        1   2.597444
19     0.039714  0.489415      19        1   2.455103
16    -0.108305  0.460418      20        1   2.441214
2     -0.275197  0.426296      21        1   2.406400
24    -0.309078 -0.066816      22        1   2.276926
8     -0.012923 -0.515990      23        1   2.198156
1      0.323023 -0.391818      24        1   2.191061
13     0.309101  0.217062      25        1   2.183358
7      0.187461  0.445874      26        1   1.991879, topic_info=              Term          Freq         Total Category  logprob  loglift
8              the  77153.000000  77153.000000  Default  10.0000  10.0000
3                a  49879.000000  49879.000000  Default   9.0000   9.0000
2             this  18399.000000  18399.000000  Default   8.0000   8.0000
152            our   9814.000000   9814.000000  Default   7.0000   7.0000
33           these   7910.000000   7910.000000  Default   6.0000   6.0000
...            ...           ...           ...      ...      ...      ...
813         market    590.781359    591.969786  Topic26  -3.1561   3.9141
677   distribution    482.105087    483.293514  Topic26  -3.3594   3.9136
111            bit    440.983743    442.172170  Topic26  -3.4485   3.9134
263    traditional    362.446027    363.634455  Topic26  -3.6447   3.9128
1276          male    355.930243    357.118671  Topic26  -3.6628   3.9128

[270 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
213      23  0.999595  "the
113      13  0.999762     -
230       9  0.999182     0
664       8  0.999057   000
386       8  0.999690     1
...     ...       ...   ...
221       7  0.999154     x
278      17  0.999079     y
122      17  0.999054  year
403      11  0.999856  your
710       5  0.997958  zero

[260 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 26, 12, 6, 1, 21, 13, 5, 7, 24, 15, 10, 16, 18, 23, 19, 11, 22, 20, 17, 3, 25, 9, 2, 14, 8])

In [71]:
validation.head(5)

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,urlurl,views,transcript,urlurl
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...


In [87]:
new_text_corpus = words.doc2bow(validation['transcript'][0].split())
print(len(new_text_corpus))
prediction = lda_model.get_document_topics(new_text_corpus)
prediction.sort(key = lambda x: x[1], reverse = True)
print(prediction)
prediction = prediction[0][0]
print("Predicted Topic: %d" % prediction)
lda_model.show_topic(3)

314
[(3, 0.13258648), (9, 0.12351883), (21, 0.104887806), (10, 0.08209654), (25, 0.074027106), (0, 0.07038231), (20, 0.052619357), (11, 0.04937543), (18, 0.037321743), (12, 0.03656352), (16, 0.026308067), (2, 0.025468623), (23, 0.023019968), (22, 0.022977022), (15, 0.02129923), (6, 0.017959973), (24, 0.01448652), (14, 0.013678219), (1, 0.01363048), (19, 0.011965354), (5, 0.011268145)]
Predicted Topic: 3


[('the', 0.8425691),
 ('value', 0.020906031),
 ('low', 0.013843658),
 ('unit', 0.01102224),
 ('next', 0.01026262),
 ('idea', 0.0081248265),
 ('maybe', 0.0067677856),
 ('pressure', 0.0065477733),
 ('specific', 0.0060585267),
 ('chain', 0.0059726792)]